# 🔬 Notebook 02: Análisis de Activaciones

## 🎯 Objetivo

Analizar en profundidad las activaciones de capas intermedias de ResNet18 para entender:
- ¿Qué patrones captura cada capa?
- ¿Cómo varían las activaciones entre capas?
- ¿Cuál es la sparsity de las activaciones?
- ¿Hay neuronas "muertas" o poco activas?
- ¿Cómo se distribuyen las activaciones?

## 📋 Contenido

1. Setup y carga del modelo
2. Extracción de activaciones en batch
3. Análisis estadístico por capa
4. Visualización de distribuciones
5. Análisis de sparsity
6. Identificación de neuronas muertas
7. Comparación entre clases
8. Conclusiones y siguientes pasos

In [ ]:
# ============================================================================
# 📦 1. Configuración de paths y imports básicos
# ============================================================================

import sys
import os
from pathlib import Path

# Detectar el directorio raíz del proyecto
current_dir = Path.cwd()
print(f"📁 Directorio actual: {current_dir}")

# Si estamos en notebooks/, subir un nivel
if 'notebooks' in str(current_dir):
    project_root = current_dir.parent
else:
    project_root = current_dir

print(f"📁 Directorio raíz del proyecto: {project_root}")

# Agregar el directorio raíz al path de Python
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Agregado al sys.path: {project_root}")

# Verificar que src/ existe
src_path = project_root / 'src'
if src_path.exists():
    print(f"✅ Directorio src encontrado: {src_path}")
else:
    print(f"❌ ERROR: Directorio src NO encontrado en {src_path}")

# Verificar que analyze_neuron.py existe
analyze_neuron_path = src_path / 'utils' / 'analyze_neuron.py'
if analyze_neuron_path.exists():
    print(f"✅ Archivo analyze_neuron.py encontrado")
else:
    print(f"❌ ERROR: analyze_neuron.py NO encontrado")

print("\n" + "="*80)

---
## 📦 1. Imports y Configuración

In [ ]:
# Imports estándar
import sys
import os
from pathlib import Path
import logging
from typing import Dict, List, Tuple, Optional
from collections import defaultdict, OrderedDict

# Librerías científicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Configuración de estilos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Agregar src al path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root / 'src'))

print(f"✅ Project root: {project_root}")
print(f"✅ Python version: {sys.version.split()[0]}")
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

---
## 🔧 2. Cargar Modelo y Dataset

Reutilizamos las clases implementadas en el Notebook 01.

In [ ]:
# Importar nuestras utilidades
from models.model_loader import ModelLoader
from utils.image_loader import ImageLoader
from utils.hooks import ActivationHook, get_layer_types

# Configuración
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 64
NUM_WORKERS = 4

print(f"🖥️  Device: {DEVICE}")
print(f"📊 Batch size: {BATCH_SIZE}")

In [ ]:
# Cargar modelo ResNet18
model_loader = ModelLoader(
    model_name='resnet18',
    pretrained=True,
    device=str(DEVICE)
)

model = model_loader.load_model()
model.eval()  # Modo evaluación

# Mostrar información del modelo
arch_info = model_loader.get_architecture_info()
print(f"\n📊 Modelo: ResNet18")
print(f"   - Parámetros totales: {arch_info['total_params']:,}")
print(f"   - Tamaño: {arch_info['model_size_mb']:.2f} MB")
print(f"   - Capas: {arch_info['num_layers']}")

In [ ]:
# Cargar dataset CIFAR-10
image_loader = ImageLoader(
    dataset_name='cifar10',
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    download=True
)

train_loader, test_loader = image_loader.get_dataloaders()
dataset_info = image_loader.get_dataset_info()

print(f"\n📦 Dataset: {dataset_info['name'].upper()}")
print(f"   - Clases: {dataset_info['num_classes']}")
print(f"   - Train samples: {dataset_info['train_samples']:,}")
print(f"   - Test samples: {dataset_info['test_samples']:,}")
print(f"   - Image size: {dataset_info['image_size']}")
print(f"\n🏷️  Clases: {', '.join(dataset_info['classes'])}")

---
## 🎯 3. Seleccionar Capas Objetivo para Análisis

Vamos a analizar capas estratégicas de ResNet18:
- **conv1**: Primera capa convolucional (features de bajo nivel)
- **layer1**: Primer bloque residual
- **layer2**: Segundo bloque residual
- **layer3**: Tercer bloque residual
- **layer4**: Cuarto bloque residual (features de alto nivel)
- **fc**: Capa fully connected final

In [ ]:
# Obtener estructura de capas del modelo
layer_types = get_layer_types(model)

print("🔍 Tipos de capas en ResNet18:")
for layer_type, layers in layer_types.items():
    print(f"   - {layer_type}: {len(layers)} capas")
    if len(layers) <= 5:
        print(f"     {layers}")

In [ ]:
# Definir capas objetivo para análisis
# Seleccionamos capas conv2d estratégicas de cada bloque
TARGET_LAYERS = [
    'relu',                    # ReLU después de conv1: 64 filtros
    'layer1.0.relu',          # ReLU del bloque 1.0
    'layer1.1.relu',          # ReLU del bloque 1.1
    'layer2.0.relu',          # ReLU del bloque 2.0
    'layer2.1.relu',          # ReLU del bloque 2.1
    'layer3.0.relu',          # ReLU del bloque 3.0
    'layer3.1.relu',          # ReLU del bloque 3.1
    'layer4.0.relu',          # ReLU del bloque 4.0
    'layer4.1.relu',          # ReLU del bloque 4.1
]

print(f"\n🎯 Capas seleccionadas para análisis: {len(TARGET_LAYERS)}")
for i, layer in enumerate(TARGET_LAYERS, 1):
    print(f"   {i}. {layer}")

---
## 🔌 4. Registrar Hooks y Extraer Activaciones

Usamos la clase `ActivationHook` para capturar activaciones de las capas seleccionadas.

In [ ]:
# Crear hook para capturar activaciones
activation_hook = ActivationHook(model, target_layers=TARGET_LAYERS)
activation_hook.register_hooks(capture_gradients=False)

print(f"✅ Hooks registrados en {len(TARGET_LAYERS)} capas")
print(f"📋 Capas monitoreadas: {activation_hook.get_layer_names()}")

In [ ]:
# Función para extraer activaciones de un batch
def extract_activations_from_batch(
    model: nn.Module,
    hook: ActivationHook,
    images: torch.Tensor,
    device: torch.device
) -> Dict[str, torch.Tensor]:
    """
    Extrae activaciones de un batch de imágenes.
    
    Args:
        model: Modelo de PyTorch
        hook: ActivationHook registrado
        images: Batch de imágenes [B, C, H, W]
        device: Device (CPU/GPU)
    
    Returns:
        Diccionario con activaciones por capa
    """
    # Limpiar activaciones previas
    hook.clear_activations()
    
    # Forward pass
    images = images.to(device)
    with torch.no_grad():
        _ = model(images)
    
    # Obtener activaciones (se mantienen en GPU para eficiencia)
    activations = hook.get_activations()
    
    return activations

print("✅ Función de extracción definida")

---
## 📊 5. Extraer Activaciones de Múltiples Batches

Para obtener estadísticas robustas, extraemos activaciones de varios batches.

In [ ]:
# Configuración de extracción
NUM_BATCHES_TO_ANALYZE = 10  # Analizar 10 batches (640 imágenes)
SAMPLE_SIZE = NUM_BATCHES_TO_ANALYZE * BATCH_SIZE

print(f"🔢 Configuración de extracción:")
print(f"   - Batches a analizar: {NUM_BATCHES_TO_ANALYZE}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Total de imágenes: {SAMPLE_SIZE}")

In [ ]:
# Estructura para acumular activaciones
# accumulated_activations[layer_name] = lista de tensors
accumulated_activations = defaultdict(list)
accumulated_labels = []

print("\n🚀 Extrayendo activaciones...")

# Iterar sobre batches del test set
with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(tqdm(test_loader, total=NUM_BATCHES_TO_ANALYZE)):
        if batch_idx >= NUM_BATCHES_TO_ANALYZE:
            break
        
        # Extraer activaciones de este batch
        batch_activations = extract_activations_from_batch(
            model, activation_hook, images, DEVICE
        )
        
        # Acumular activaciones (mantener en GPU por ahora)
        for layer_name, activation in batch_activations.items():
            accumulated_activations[layer_name].append(activation.detach())
        
        # Acumular labels
        accumulated_labels.extend(labels.tolist())

print(f"\n✅ Activaciones extraídas de {len(accumulated_labels)} imágenes")
print(f"📊 Capas analizadas: {len(accumulated_activations)}")

In [ ]:
# Concatenar activaciones de todos los batches
# Ahora cada capa tendrá un tensor de shape [total_images, ...]
print("🔄 Concatenando activaciones...")

concatenated_activations = {}
for layer_name, activation_list in accumulated_activations.items():
    concatenated_activations[layer_name] = torch.cat(activation_list, dim=0)

print("\n📐 Shapes de activaciones concatenadas:")
for layer_name, activation in concatenated_activations.items():
    print(f"   {layer_name}: {list(activation.shape)}")

# Convertir labels a numpy
labels_array = np.array(accumulated_labels)
print(f"\n🏷️  Labels shape: {labels_array.shape}")

---
## 📈 6. Análisis Estadístico de Activaciones

Calculamos estadísticas clave para cada capa:
- Media y desviación estándar
- Valores mínimos y máximos
- Sparsity (porcentaje de activaciones == 0)
- Número de neuronas activas

In [ ]:
# Función para calcular estadísticas detalladas
def compute_activation_statistics(
    activations: Dict[str, torch.Tensor]
) -> pd.DataFrame:
    """
    Calcula estadísticas completas de activaciones por capa.
    
    Args:
        activations: Diccionario con activaciones por capa
    
    Returns:
        DataFrame con estadísticas por capa
    """
    stats_list = []
    
    for layer_name, activation in activations.items():
        # Convertir a numpy para cálculos (mover a CPU si está en GPU)
        act_np = activation.cpu().numpy()
        
        # Calcular estadísticas
        stats = {
            'layer': layer_name,
            'shape': str(list(activation.shape)),
            'total_elements': activation.numel(),
            'mean': float(act_np.mean()),
            'std': float(act_np.std()),
            'min': float(act_np.min()),
            'max': float(act_np.max()),
            'median': float(np.median(act_np.flatten())),
            'q25': float(np.percentile(act_np.flatten(), 25)),
            'q75': float(np.percentile(act_np.flatten(), 75)),
            'sparsity_%': float((act_np == 0).sum() / act_np.size * 100),
            'active_neurons': int((act_np != 0).any(axis=(0, 2, 3)).sum() if len(act_np.shape) == 4 else (act_np != 0).any(axis=0).sum()),
            'total_neurons': activation.shape[1] if len(activation.shape) == 4 else activation.shape[-1],
        }
        
        stats_list.append(stats)
    
    return pd.DataFrame(stats_list)

# Calcular estadísticas
stats_df = compute_activation_statistics(concatenated_activations)

print("\n📊 Estadísticas de Activaciones por Capa:")
print("=" * 80)
display(stats_df)

In [ ]:
# Visualizar estadísticas clave
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Media de activaciones por capa
axes[0, 0].bar(range(len(stats_df)), stats_df['mean'], color='steelblue')
axes[0, 0].set_xticks(range(len(stats_df)))
axes[0, 0].set_xticklabels(stats_df['layer'], rotation=45, ha='right')
axes[0, 0].set_ylabel('Media')
axes[0, 0].set_title('Media de Activaciones por Capa')
axes[0, 0].grid(alpha=0.3)

# 2. Desviación estándar por capa
axes[0, 1].bar(range(len(stats_df)), stats_df['std'], color='coral')
axes[0, 1].set_xticks(range(len(stats_df)))
axes[0, 1].set_xticklabels(stats_df['layer'], rotation=45, ha='right')
axes[0, 1].set_ylabel('Desviación Estándar')
axes[0, 1].set_title('Desviación Estándar por Capa')
axes[0, 1].grid(alpha=0.3)

# 3. Sparsity por capa
axes[1, 0].bar(range(len(stats_df)), stats_df['sparsity_%'], color='mediumseagreen')
axes[1, 0].set_xticks(range(len(stats_df)))
axes[1, 0].set_xticklabels(stats_df['layer'], rotation=45, ha='right')
axes[1, 0].set_ylabel('Sparsity (%)')
axes[1, 0].set_title('Sparsity de Activaciones por Capa')
axes[1, 0].grid(alpha=0.3)

# 4. Rango (max - min) por capa
activation_range = stats_df['max'] - stats_df['min']
axes[1, 1].bar(range(len(stats_df)), activation_range, color='orchid')
axes[1, 1].set_xticks(range(len(stats_df)))
axes[1, 1].set_xticklabels(stats_df['layer'], rotation=45, ha='right')
axes[1, 1].set_ylabel('Rango')
axes[1, 1].set_title('Rango de Activaciones (Max - Min) por Capa')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

---
## 📊 7. Distribuciones de Activaciones

Visualizamos la distribución de activaciones para entender mejor su comportamiento.

In [ ]:
# Seleccionar algunas capas representativas para visualizar distribuciones
LAYERS_TO_PLOT = ['relu', 'layer1.1.relu', 'layer2.1.relu', 'layer3.1.relu', 'layer4.1.relu']

print(f"📊 Capas seleccionadas para visualizar distribuciones: {len(LAYERS_TO_PLOT)}")
for layer in LAYERS_TO_PLOT:
    if layer in concatenated_activations:
        print(f"   ✅ {layer}")
    else:
        print(f"   ❌ {layer} - NO ENCONTRADA")
        
fig, axes = plt.subplots(len(LAYERS_TO_PLOT), 2, figsize=(15, 4 * len(LAYERS_TO_PLOT)))

for idx, layer_name in enumerate(LAYERS_TO_PLOT):
    if layer_name not in concatenated_activations:
        continue
    
    activation = concatenated_activations[layer_name].cpu().numpy()
    activation_flat = activation.flatten()
    
    # Histograma
    axes[idx, 0].hist(activation_flat, bins=100, alpha=0.7, color='steelblue', edgecolor='black')
    axes[idx, 0].set_xlabel('Valor de Activación')
    axes[idx, 0].set_ylabel('Frecuencia')
    axes[idx, 0].set_title(f'Distribución de Activaciones - {layer_name}')
    axes[idx, 0].set_yscale('log')  # Escala log para mejor visualización
    axes[idx, 0].grid(alpha=0.3)
    
    # Box plot
    # Tomar muestra para box plot (demasiados puntos pueden ser lentos)
    sample_size = min(10000, len(activation_flat))
    sample_indices = np.random.choice(len(activation_flat), sample_size, replace=False)
    activation_sample = activation_flat[sample_indices]
    
    axes[idx, 1].boxplot(activation_sample, vert=True, patch_artist=True,
                         boxprops=dict(facecolor='lightblue', alpha=0.7))
    axes[idx, 1].set_ylabel('Valor de Activación')
    axes[idx, 1].set_title(f'Box Plot - {layer_name}')
    axes[idx, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Observaciones:")
print("   - Capas tempranas tienden a tener distribuciones más amplias")
print("   - Capas profundas pueden mostrar mayor sparsity")
print("   - La escala logarítmica ayuda a visualizar valores pequeños")

---
## 🔍 8. Análisis de Sparsity y Neuronas Muertas

Identificamos neuronas que nunca se activan (o muy raramente).

In [ ]:
# Función para identificar neuronas muertas
def find_dead_neurons_detailed(
    activations: Dict[str, torch.Tensor],
    threshold: float = 1e-6
) -> Dict[str, Dict]:
    """
    Identifica neuronas "muertas" (que nunca se activan significativamente).
    
    Args:
        activations: Diccionario con activaciones por capa
        threshold: Umbral para considerar una neurona como activa
    
    Returns:
        Diccionario con información de neuronas muertas por capa
    """
    dead_neurons_info = {}
    
    for layer_name, activation in activations.items():
        act_np = activation.cpu().numpy()
        
        # Para capas conv: [B, C, H, W] -> analizar por canal
        if len(act_np.shape) == 4:
            # Máximo de cada neurona (canal) a través de batch y spatial dims
            max_per_neuron = act_np.max(axis=(0, 2, 3))
        # Para capas fc: [B, N] -> analizar por neurona
        else:
            max_per_neuron = act_np.max(axis=0)
        
        # Identificar neuronas muertas
        dead_mask = max_per_neuron <= threshold
        dead_indices = np.where(dead_mask)[0]
        
        total_neurons = len(max_per_neuron)
        dead_count = len(dead_indices)
        dead_percentage = (dead_count / total_neurons * 100) if total_neurons > 0 else 0
        
        dead_neurons_info[layer_name] = {
            'total_neurons': total_neurons,
            'dead_neurons': dead_count,
            'dead_percentage': dead_percentage,
            'dead_indices': dead_indices.tolist(),
            'max_activations': max_per_neuron.tolist()
        }
    
    return dead_neurons_info

# Analizar neuronas muertas
dead_neurons_info = find_dead_neurons_detailed(concatenated_activations, threshold=1e-6)

print("\n🔍 Análisis de Neuronas Muertas:")
print("=" * 80)
for layer_name, info in dead_neurons_info.items():
    print(f"\n{layer_name}:")
    print(f"   Total de neuronas: {info['total_neurons']}")
    print(f"   Neuronas muertas: {info['dead_neurons']} ({info['dead_percentage']:.2f}%)")
    if info['dead_neurons'] > 0:
        print(f"   Índices: {info['dead_indices'][:10]}{'...' if len(info['dead_indices']) > 10 else ''}")

In [ ]:
# Ver la neurona muerta #38 de conv1
dead_filter_weights = model.conv1.weight[38].detach().cpu().numpy()
# Shape: [3, 7, 7] (canal RGB, kernel 7x7)

plt.imshow(dead_filter_weights.transpose(1, 2, 0))
plt.title('Filtro #38 de conv1 (MUERTO)')
plt.show()

# Comparar con uno activo
active_filter = model.conv1.weight[0].detach().cpu().numpy()
plt.imshow(active_filter.transpose(1, 2, 0))
plt.title('Filtro #0 de conv1 (ACTIVO)')
plt.show()

In [ ]:
# Visualizar porcentaje de neuronas muertas por capa
layers = list(dead_neurons_info.keys())
dead_percentages = [dead_neurons_info[layer]['dead_percentage'] for layer in layers]

plt.figure(figsize=(12, 6))
bars = plt.bar(range(len(layers)), dead_percentages, color='crimson', alpha=0.7, edgecolor='black')
plt.xticks(range(len(layers)), layers, rotation=45, ha='right')
plt.ylabel('Porcentaje de Neuronas Muertas (%)')
plt.title('Neuronas Muertas por Capa (threshold = 1e-6)')
plt.grid(alpha=0.3, axis='y')

# Añadir valores en las barras
for i, bar in enumerate(bars):
    height = bar.get_height()
    if height > 0:
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n💡 Interpretación:")
print("   - Un alto % de neuronas muertas puede indicar:")
print("     1. Over-parametrización del modelo")
print("     2. ReLU 'muriendo' (dying ReLU problem)")
print("     3. Neuronas especializadas que solo se activan con ciertos inputs")

---
## 🎨 9. Comparación de Activaciones entre Clases

¿Las activaciones varían según la clase de la imagen?

In [ ]:
# Función para calcular estadísticas por clase
def compute_class_wise_statistics(
    activations: Dict[str, torch.Tensor],
    labels: np.ndarray,
    class_names: List[str]
) -> Dict[str, pd.DataFrame]:
    """
    Calcula estadísticas de activaciones separadas por clase.
    
    Args:
        activations: Diccionario con activaciones por capa
        labels: Array con labels de cada imagen
        class_names: Lista con nombres de clases
    
    Returns:
        Diccionario con DataFrames de estadísticas por clase
    """
    class_stats = {}
    
    for layer_name, activation in activations.items():
        act_np = activation.cpu().numpy()
        
        # Calcular estadísticas por clase
        stats_by_class = []
        
        for class_id, class_name in enumerate(class_names):
            # Filtrar activaciones de esta clase
            class_mask = labels == class_id
            if class_mask.sum() == 0:
                continue
            
            class_activations = act_np[class_mask]
            
            stats = {
                'class_id': class_id,
                'class_name': class_name,
                'num_samples': class_mask.sum(),
                'mean': float(class_activations.mean()),
                'std': float(class_activations.std()),
                'median': float(np.median(class_activations)),
                'sparsity_%': float((class_activations == 0).sum() / class_activations.size * 100)
            }
            stats_by_class.append(stats)
        
        class_stats[layer_name] = pd.DataFrame(stats_by_class)
    
    return class_stats

# Calcular estadísticas por clase
class_wise_stats = compute_class_wise_statistics(
    concatenated_activations,
    labels_array,
    dataset_info['classes']
)

print("✅ Estadísticas por clase calculadas")

In [ ]:
# Visualizar estadísticas de una capa específica por clase
LAYER_TO_ANALYZE = 'layer4.1.relu'  # Última capa convolucional

if LAYER_TO_ANALYZE in class_wise_stats:
    df = class_wise_stats[LAYER_TO_ANALYZE]
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Media por clase
    axes[0].bar(df['class_name'], df['mean'], color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Clase')
    axes[0].set_ylabel('Media de Activaciones')
    axes[0].set_title(f'Media de Activaciones por Clase - {LAYER_TO_ANALYZE}')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(alpha=0.3, axis='y')
    
    # Sparsity por clase
    axes[1].bar(df['class_name'], df['sparsity_%'], color='coral', alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Clase')
    axes[1].set_ylabel('Sparsity (%)')
    axes[1].set_title(f'Sparsity por Clase - {LAYER_TO_ANALYZE}')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Estadísticas por Clase - {LAYER_TO_ANALYZE}:")
    print("=" * 80)
    display(df)
else:
    print(f"❌ Capa {LAYER_TO_ANALYZE} no encontrada")

---
## 📊 10. Heatmap de Activaciones Promedio

Visualizamos la activación promedio de cada neurona para identificar patrones.

In [ ]:
# Seleccionar una capa para visualizar heatmap
LAYER_FOR_HEATMAP = 'layer3.1.relu'

if LAYER_FOR_HEATMAP in concatenated_activations:
    activation = concatenated_activations[LAYER_FOR_HEATMAP].cpu().numpy()
    # Shape: [num_images, num_channels, H, W]
    
    # Calcular activación promedio por canal a través de spatial dimensions
    # Resultado: [num_images, num_channels]
    avg_per_channel = activation.mean(axis=(2, 3))
    
    # Seleccionar subset de imágenes para visualización (primeras 50)
    num_images_to_plot = min(50, avg_per_channel.shape[0])
    avg_per_channel_subset = avg_per_channel[:num_images_to_plot]

    print(f"Este gráfico visualiza cuánto se activa cada neurona de la capa layer3.1.relu ")
    print(f"para cada una de las primeras 50 imágenes del dataset.")
    # Crear heatmap
    plt.figure(figsize=(15, 10))
    sns.heatmap(
        avg_per_channel_subset.T,
        cmap='YlOrRd',
        xticklabels=5,
        yticklabels=10,
        cbar_kws={'label': 'Activación Promedio'}
    )
    plt.xlabel('Índice de Imagen')
    plt.ylabel('Canal (Neurona)')
    plt.title(f'Heatmap de Activaciones Promedio - {LAYER_FOR_HEATMAP}\n(Primeras {num_images_to_plot} imágenes)')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Interpretación del Heatmap:")
    print("   - Filas = canales/neuronas de la capa")
    print("   - Columnas = imágenes procesadas")
    print("   - Colores cálidos = activaciones altas")
    print("   - Color amarillo claro = Activación ≈ 0")
    print("   - Patrones horizontales = neuronas especializadas")
    print("   - Patrones verticales = imágenes con características similares")

    print("--------------------------")
    print("** HEATMAP SALUDABLE **")
    print("   - Mayoría amarillo (sparsity esperada)")
    print("   - Puntos rojos dispersos (selectividad)")
    print("   - Sin filas completamente rojas (no overfitting)")
    print("   - Sin filas completamente rojas (no overfitting)")
    print("--------------------------")
    print("** HEATMAP PROBLEMÁTICO **")
    print("   - Todo naranja/rojo (sin sparsity, ReLU no funciona)")
    print("   - Filas completamente rojas (neurona siempre activa, no selectiva)")
    print("   - Columnas completamente amarillas (imagen no reconocida)")
    print("   - Patrón de cuadrícula (neuronas redundantes)")

else:
    print(f"❌ Capa {LAYER_FOR_HEATMAP} no encontrada")

In [ ]:
# Nueva celda para investigar
print("="*80)
print("🔍 INVESTIGACIÓN: ¿Qué neuronas se activaron FUERTE con imagen #5?")
print("="*80)

IMAGE_INDEX = 5
LAYER_TO_INSPECT = 'layer3.1.relu'

if LAYER_TO_INSPECT in concatenated_activations:
    # Obtener activaciones de TODAS las neuronas para imagen #5
    all_neurons_img5 = concatenated_activations[LAYER_TO_INSPECT][IMAGE_INDEX]
    # Shape: [256 neuronas, 2, 2]
    
    # Calcular activación promedio espacial de cada neurona
    avg_per_neuron = all_neurons_img5.mean(dim=(1, 2)).cpu().numpy()
    # Shape: [256]
    
    # Ordenar neuronas por activación (de mayor a menor)
    neuron_indices = np.argsort(avg_per_neuron)[::-1]
    activations_sorted = avg_per_neuron[neuron_indices]
    
    # Top 10 neuronas más activas
    print(f"\n🏆 TOP 10 NEURONAS MÁS ACTIVAS para imagen #5 (frog):")
    print("="*80)
    for i in range(10):
        neuron_id = neuron_indices[i]
        activation = activations_sorted[i]
        print(f"   #{i+1}. Neurona #{neuron_id:3d}: Activación = {activation:.4f}")
    
    # ¿Dónde está la neurona #85?
    rank_85 = np.where(neuron_indices == 85)[0][0] + 1
    activation_85 = avg_per_neuron[85]
    
    print(f"\n🎯 Neurona #85 (la que analizamos):")
    print(f"   - Ranking: #{rank_85}/256")
    print(f"   - Activación: {activation_85:.4f}")
    
    if rank_85 <= 50:
        print(f"   ✅ Está en el top 50 (relativamente activa)")
    else:
        print(f"   ⚠️  NO está en el top 50 (moderadamente activa)")
    
    # Visualizar distribución
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histograma de activaciones de todas las neuronas
    axes[0].hist(avg_per_neuron, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
    axes[0].axvline(activation_85, color='red', linestyle='--', linewidth=2,
                   label=f'Neurona #85 (rank #{rank_85})')
    axes[0].set_xlabel('Activación Promedio')
    axes[0].set_ylabel('Frecuencia (número de neuronas)')
    axes[0].set_title(f'Distribución de Activaciones de las 256 Neuronas\nImagen #5 (frog)')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Bar plot del top 20
    top_20_indices = neuron_indices[:20]
    top_20_activations = activations_sorted[:20]
    
    colors = ['red' if idx == 85 else 'steelblue' for idx in top_20_indices]
    bars = axes[1].bar(range(20), top_20_activations, color=colors, alpha=0.7, edgecolor='black')
    axes[1].set_xticks(range(20))
    axes[1].set_xticklabels([f'#{idx}' for idx in top_20_indices], rotation=45, ha='right')
    axes[1].set_xlabel('Neurona ID')
    axes[1].set_ylabel('Activación')
    axes[1].set_title('Top 20 Neuronas Más Activas\n(Rojo = Neurona #85)')
    axes[1].grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 EXPLICACIÓN DE LA 'RAYA ROJA' EN EL HEATMAP:")
    print("="*80)
    print(f"   - El heatmap muestra TODAS las 256 neuronas")
    print(f"   - Para imagen #5, hay {(avg_per_neuron > 1.0).sum()} neuronas con activación > 1.0")
    print(f"   - Estas neuronas 'rojas' crearon la raya visible en la columna")
    print(f"   - La neurona #85 (rank #{rank_85}) contribuyó moderadamente")
    print(f"   - Las neuronas del top 10 son las que realmente causaron la raya roja")
    
else:
    print(f"❌ Capa no encontrada")

In [ ]:
# ============================================================================
# 🔬 ANÁLISIS DETALLADO: Neurona específica + Imagen específica
# ============================================================================

# Configuración
IMAGE_INDEX = 5          # Índice de la imagen a analizar
NEURON_INDEX = 83        # Índice de la neurona que queremos investigar
LAYER_TO_INSPECT = 'layer3.1.relu'  # Capa donde está la neurona

print("="*80)
print("🔍 ANÁLISIS DETALLADO DE ACTIVACIÓN")
print("="*80)
print(f"📊 Capa: {LAYER_TO_INSPECT}")
print(f"🧠 Neurona: #{NEURON_INDEX}")
print(f"🖼️  Imagen: #{IMAGE_INDEX}")
print("="*80)

In [ ]:
# ============================================================================
# 1️⃣ VISUALIZAR LA IMAGEN
# ============================================================================

# Obtener la imagen específica del test set
test_images, test_labels = next(iter(test_loader))

# Extraer imagen y label
image = test_images[IMAGE_INDEX]
label = test_labels[IMAGE_INDEX]
class_name = dataset_info['classes'][label]

# Denormalizar para visualización
image_denorm = image_loader.denormalize_image(image)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Imagen original
axes[0].imshow(image_denorm)
axes[0].set_title(f'Imagen #{IMAGE_INDEX}\nClase: {class_name.upper()}', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Imagen con información
axes[1].imshow(image_denorm)
axes[1].set_title(f'Dimensiones: {image.shape}\nNormalización: ImageNet', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ Imagen cargada correctamente")
print(f"   - Clase real: {class_name}")
print(f"   - Label ID: {label}")
print(f"   - Shape original: {image.shape}")

In [ ]:
# ============================================================================
# 2️⃣ EXTRAER ACTIVACIÓN DE LA NEURONA PARA ESTA IMAGEN
# ============================================================================

# Verificar que tenemos las activaciones
if LAYER_TO_INSPECT in concatenated_activations:
    # Obtener activaciones de la capa completa
    layer_activations = concatenated_activations[LAYER_TO_INSPECT]
    # Shape esperado: [num_images, num_channels, H, W]
    
    # Extraer activación de la imagen específica
    image_activation = layer_activations[IMAGE_INDEX]  # Shape: [num_channels, H, W]
    
    # Extraer activación de la neurona específica
    neuron_activation = image_activation[NEURON_INDEX]  # Shape: [H, W]
    
    print(f"\n📊 INFORMACIÓN DE ACTIVACIONES:")
    print(f"   - Shape de la capa completa: {list(layer_activations.shape)}")
    print(f"   - Shape para imagen #{IMAGE_INDEX}: {list(image_activation.shape)}")
    print(f"   - Shape de neurona #{NEURON_INDEX}: {list(neuron_activation.shape)}")
    
    # Convertir a numpy para análisis
    neuron_activation_np = neuron_activation.cpu().numpy()
    
    # Estadísticas de la activación
    print(f"\n🔢 ESTADÍSTICAS DE LA ACTIVACIÓN:")
    print(f"   - Valor máximo: {neuron_activation_np.max():.4f}")
    print(f"   - Valor mínimo: {neuron_activation_np.min():.4f}")
    print(f"   - Valor promedio: {neuron_activation_np.mean():.4f}")
    print(f"   - Desviación estándar: {neuron_activation_np.std():.4f}")
    print(f"   - Sparsity: {(neuron_activation_np == 0).sum() / neuron_activation_np.size * 100:.1f}%")
    
    # Calcular activación promedio espacial (el valor que vimos en el heatmap)
    avg_activation = neuron_activation_np.mean()
    print(f"\n🎯 ACTIVACIÓN PROMEDIO (valor del heatmap): {avg_activation:.4f}")
    
else:
    print(f"❌ ERROR: Capa '{LAYER_TO_INSPECT}' no encontrada en las activaciones")

In [ ]:
# ============================================================================
# 3️⃣ VISUALIZAR EL MAPA DE ACTIVACIÓN ESPACIAL
# ============================================================================

if LAYER_TO_INSPECT in concatenated_activations:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. Imagen original
    axes[0].imshow(image_denorm)
    axes[0].set_title(f'Imagen Original\n{class_name.upper()}', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # 2. Mapa de activación (heatmap)
    im = axes[1].imshow(neuron_activation_np, cmap='hot', interpolation='nearest')
    axes[1].set_title(f'Activación de Neurona #{NEURON_INDEX}\n{LAYER_TO_INSPECT}', 
                      fontsize=12, fontweight='bold')
    axes[1].set_xlabel(f'Ancho espacial (shape: {neuron_activation_np.shape[1]})')
    axes[1].set_ylabel(f'Alto espacial (shape: {neuron_activation_np.shape[0]})')
    plt.colorbar(im, ax=axes[1], label='Intensidad de activación')
    
    # 3. Superposición (overlay)
    # Redimensionar activación al tamaño de la imagen original
    from scipy.ndimage import zoom
    zoom_factors = (image_denorm.shape[0] / neuron_activation_np.shape[0],
                    image_denorm.shape[1] / neuron_activation_np.shape[1])
    activation_resized = zoom(neuron_activation_np, zoom_factors, order=1)
    
    # Normalizar para overlay
    activation_norm = (activation_resized - activation_resized.min()) / (activation_resized.max() - activation_resized.min() + 1e-8)
    
    # Mostrar imagen con overlay
    axes[2].imshow(image_denorm)
    axes[2].imshow(activation_norm, cmap='hot', alpha=0.5)  # Alpha para transparencia
    axes[2].set_title(f'Superposición: ¿Dónde se activa?\n(Rojo = alta activación)', 
                      fontsize=12, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 INTERPRETACIÓN DEL MAPA DE ACTIVACIÓN:")
    print("   - Mapa central: Muestra la activación espacial (dónde 'mira' la neurona)")
    print("   - Mapa derecho: Superposición sobre la imagen original")
    print("   - Zonas rojas/brillantes = La neurona se activó fuertemente ahí")
    print("   - Zonas oscuras = La neurona no detectó nada interesante")

In [ ]:
# ============================================================================
# 4️⃣ ANÁLISIS DE SESGO ESPACIAL
# ============================================================================

from src.utils.analyze_neuron import analyze_spatial_bias, visualize_spatial_bias

# Analizar sesgo espacial de la neurona #83
spatial_results = analyze_spatial_bias(
    neuron_index=83,
    layer_name='layer3.1.relu',
    concatenated_activations=concatenated_activations,
    num_samples=50,
    verbose=True
)

# Visualizar resultados
visualize_spatial_bias(spatial_results)

# Acceso rápido a resultados
print(f"\n🎯 Resumen:")
print(f"   Sesgo horizontal: {spatial_results['horizontal_bias']['bias_type']}")
print(f"   Sesgo vertical: {spatial_results['vertical_bias']['bias_type']}")
print(f"   Sesgo dominante: {spatial_results['dominant_bias']}")

In [ ]:
# ============================================================================
# 5️⃣ ANÁLISIS POR CLASE: ¿Esta neurona es selectiva a alguna clase?
# ============================================================================

from src.utils.analyze_neuron import analyze_class_selectivity, visualize_class_selectivity

# Analizar selectividad por clase
selectivity_results = analyze_class_selectivity(
    neuron_index=83,
    layer_name='layer3.1.relu',
    concatenated_activations=concatenated_activations,
    labels=labels_array,
    class_names=dataset_info['classes'],
    num_samples=50,
    verbose=True
)

# Visualizar resultados
visualize_class_selectivity(selectivity_results)

# Acceso programático a resultados
print(f"\n🎯 Resumen rápido:")
print(f"   Clase preferida: {selectivity_results['selectivity']['top_class']}")
print(f"   Nivel de selectividad: {selectivity_results['selectivity']['level']}")
print(f"   Ratio: {selectivity_results['selectivity']['activation_ratio']:.2f}x")

In [ ]:
# ============================================================================
# 6️⃣ ANÁLISIS DE TEXTURAS: Función modular
# ============================================================================

from src.utils.analyze_neuron import analyze_neuron_texture_and_features, visualize_texture_analysis

# Llamar directamente sin crear instancia
results = analyze_neuron_texture_and_features(
    neuron_index=83,
    image_index=5,
    layer_name='layer3.1.relu',
    concatenated_activations=concatenated_activations,
    test_loader=test_loader,
    image_loader=image_loader,
    dataset_info=dataset_info,
    verbose=True
)

# Visualizar resultados
visualize_texture_analysis(results)

# Acceder a resultados
print(f"\n🎯 Acceso rápido a resultados:")
print(f"   Feature principal: {results['correlations']['top_feature']}")
print(f"   Correlación: {results['correlations']['top_correlation']:.3f}")
print(f"   Color dominante: {results['color_analysis']['dominant_channel']}")
print(f"   Tipo de textura: {results['texture_analysis']['type']}")
print(f"   Complejidad de forma: {results['shape_analysis']['complexity']}")

---
## 🔬 11. Análisis de Evolución de Activaciones a través de Capas

¿Cómo cambian las activaciones a medida que avanzamos en la red?

In [ ]:
# Extraer métricas clave de todas las capas para visualizar evolución
layer_names = list(concatenated_activations.keys())
layer_means = []
layer_stds = []
layer_sparsities = []
layer_ranges = []

for layer_name in layer_names:
    act_np = concatenated_activations[layer_name].cpu().numpy()
    
    layer_means.append(act_np.mean())
    layer_stds.append(act_np.std())
    layer_sparsities.append((act_np == 0).sum() / act_np.size * 100)
    layer_ranges.append(act_np.max() - act_np.min())

# Crear gráficos de evolución
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Evolución de la media
axes[0, 0].plot(range(len(layer_names)), layer_means, marker='o', linewidth=2, markersize=8, color='steelblue')
axes[0, 0].set_xticks(range(len(layer_names)))
axes[0, 0].set_xticklabels(layer_names, rotation=45, ha='right', fontsize=8)
axes[0, 0].set_ylabel('Media')
axes[0, 0].set_title('Evolución de la Media de Activaciones')
axes[0, 0].grid(alpha=0.3)

# 2. Evolución de la desviación estándar
axes[0, 1].plot(range(len(layer_names)), layer_stds, marker='s', linewidth=2, markersize=8, color='coral')
axes[0, 1].set_xticks(range(len(layer_names)))
axes[0, 1].set_xticklabels(layer_names, rotation=45, ha='right', fontsize=8)
axes[0, 1].set_ylabel('Desviación Estándar')
axes[0, 1].set_title('Evolución de la Desviación Estándar')
axes[0, 1].grid(alpha=0.3)

# 3. Evolución de la sparsity
axes[1, 0].plot(range(len(layer_names)), layer_sparsities, marker='^', linewidth=2, markersize=8, color='mediumseagreen')
axes[1, 0].set_xticks(range(len(layer_names)))
axes[1, 0].set_xticklabels(layer_names, rotation=45, ha='right', fontsize=8)
axes[1, 0].set_ylabel('Sparsity (%)')
axes[1, 0].set_title('Evolución de la Sparsity')
axes[1, 0].grid(alpha=0.3)

# 4. Evolución del rango
axes[1, 1].plot(range(len(layer_names)), layer_ranges, marker='D', linewidth=2, markersize=8, color='orchid')
axes[1, 1].set_xticks(range(len(layer_names)))
axes[1, 1].set_xticklabels(layer_names, rotation=45, ha='right', fontsize=8)
axes[1, 1].set_ylabel('Rango (Max - Min)')
axes[1, 1].set_title('Evolución del Rango de Activaciones')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# ---- Análisis automático de tendencias
print("\n💡 Observaciones sobre la Evolución:")
print("=" * 80)

# 1. MEDIA
mean_values = [stats_df.loc[stats_df['layer'] == layer, 'mean'].values[0] 
               for layer in layer_names]
mean_start = mean_values[0]
mean_end = mean_values[-1]
mean_max = max(mean_values)
mean_max_layer = layer_names[mean_values.index(mean_max)]
mean_min = min(mean_values)
mean_min_layer = layer_names[mean_values.index(mean_min)]

if mean_end > mean_start:
    mean_trend = "AUMENTA"
    mean_emoji = "📈"
else:
    mean_trend = "DISMINUYE"
    mean_emoji = "📉"

print(f"\n1️⃣  MEDIA: {mean_emoji} {mean_trend} con la profundidad")
print(f"   • Inicial (relu):     {mean_start:.3f}")
print(f"   • Final (layer4.1):   {mean_end:.3f}")
print(f"   • Cambio total:       {mean_end - mean_start:+.3f} ({((mean_end/mean_start - 1)*100):+.1f}%)")
print(f"   • Pico MÁXIMO:        {mean_max:.3f} en {mean_max_layer}")
print(f"   • Valle MÍNIMO:       {mean_min:.3f} en {mean_min_layer}")
print(f"   📊 Interpretación: ", end="")
if mean_max_layer == 'layer1.1.relu':
    print("Pico en layer1.1 indica features ricos antes del downsampling.")
else:
    print(f"El pico en {mean_max_layer} es inesperado, revisar.")

# 2. DESVIACIÓN ESTÁNDAR
std_values = [stats_df.loc[stats_df['layer'] == layer, 'std'].values[0] 
              for layer in layer_names]
std_start = std_values[0]
std_end = std_values[-1]
std_max = max(std_values)
std_max_layer = layer_names[std_values.index(std_max)]

if std_end > std_start * 1.5:
    std_trend = "SE VUELVEN MUCHO MÁS VARIABLES"
    std_emoji = "📊📊"
elif std_end > std_start:
    std_trend = "aumentan ligeramente"
    std_emoji = "📊"
else:
    std_trend = "se mantienen estables"
    std_emoji = "➡️"

print(f"\n2️⃣  DESVIACIÓN ESTÁNDAR: {std_emoji} {std_trend}")
print(f"   • Inicial:            {std_start:.3f}")
print(f"   • Final:              {std_end:.3f}")
print(f"   • Incremento:         {std_end - std_start:+.3f} ({((std_end/std_start - 1)*100):+.1f}%)")
print(f"   • Máximo:             {std_max:.3f} en {std_max_layer}")
print(f"   📊 Interpretación: ", end="")
if std_max_layer == layer_names[-1]:
    print("Variabilidad máxima en última capa → neuronas ultra-especializadas ✅")
else:
    print(f"Alta variabilidad en {std_max_layer} indica respuestas diversas en esa capa.")

# 3. SPARSITY
sparsity_values = [stats_df.loc[stats_df['layer'] == layer, 'sparsity_%'].values[0] 
                   for layer in layer_names]
sparsity_start = sparsity_values[0]
sparsity_end = sparsity_values[-1]
sparsity_max = max(sparsity_values)
sparsity_max_layer = layer_names[sparsity_values.index(sparsity_max)]
sparsity_min = min(sparsity_values)
sparsity_min_layer = layer_names[sparsity_values.index(sparsity_min)]

if sparsity_end > sparsity_start + 10:
    sparsity_trend = "AUMENTA SIGNIFICATIVAMENTE"
    sparsity_emoji = "🔺"
elif sparsity_end > sparsity_start:
    sparsity_trend = "aumenta moderadamente"
    sparsity_emoji = "↗️"
else:
    sparsity_trend = "se mantiene estable"
    sparsity_emoji = "➡️"

print(f"\n3️⃣  SPARSITY: {sparsity_emoji} {sparsity_trend} en capas profundas")
print(f"   • Inicial:            {sparsity_start:.1f}%")
print(f"   • Final:              {sparsity_end:.1f}%")
print(f"   • Incremento:         {sparsity_end - sparsity_start:+.1f} puntos")
print(f"   • Máximo:             {sparsity_max:.1f}% en {sparsity_max_layer}")
print(f"   • Mínimo:             {sparsity_min:.1f}% en {sparsity_min_layer}")
print(f"   📊 Interpretación: ", end="")
if sparsity_max > 50:
    print(f"¡Más del 50% de ceros en {sparsity_max_layer}! Neuronas MUY selectivas ✅")
else:
    print(f"Sparsity moderada indica balance entre activación y selectividad.")

# 4. RANGO
range_values = [stats_df.loc[stats_df['layer'] == layer, 'max'].values[0] - 
                stats_df.loc[stats_df['layer'] == layer, 'min'].values[0]
                for layer in layer_names]
range_start = range_values[0]
range_end = range_values[-1]
range_max = max(range_values)
range_max_layer = layer_names[range_values.index(range_max)]
range_min = min(range_values)

if range_end > range_start * 2:
    range_trend = "SE EXPANDEN DRAMÁTICAMENTE"
    range_emoji = "💥💥"
elif range_end > range_start * 1.2:
    range_trend = "se expanden"
    range_emoji = "📈"
else:
    range_trend = "se comprimen"
    range_emoji = "📉"

print(f"\n4️⃣  RANGO: {range_emoji} Los valores {range_trend}")
print(f"   • Inicial:            {range_start:.2f}")
print(f"   • Final:              {range_end:.2f}")
print(f"   • Multiplicador:      {range_end/range_start:.2f}x")
print(f"   • Máximo:             {range_max:.2f} en {range_max_layer}")
print(f"   • Mínimo:             {range_min:.2f}")
print(f"   📊 Interpretación: ", end="")
if range_end > 10:
    print(f"Rango >10 en última capa → activaciones EXTREMAS cuando detectan algo 🔥")
else:
    print(f"Rango moderado indica activaciones controladas.")

# RESUMEN FINAL
print("\n" + "=" * 80)
print("🎯 RESUMEN EJECUTIVO:")
print("=" * 80)
print(f"""
ResNet18 muestra un patrón claro de ESPECIALIZACIÓN PROGRESIVA:

✅ Capas tempranas (layer1):
   - Baja sparsity ({sparsity_min:.1f}%) → muchas neuronas activas
   - Variabilidad moderada → detectan features generales
   
⚠️  Transiciones entre bloques:
   - Caídas en media por downsampling
   - Saltos en sparsity (+10-20 puntos)
   
🔥 Última capa (layer4.1):
   - Alta sparsity ({sparsity_end:.1f}%) → neuronas selectivas
   - Alta variabilidad (std={std_end:.2f}) → respuestas diversas
   - Rango explosivo ({range_end:.1f}) → activaciones extremas
   
💡 Conclusión: El modelo funciona correctamente con neuronas cada vez
   más especializadas en features de alto nivel.
""")
print("=" * 80)

---
## 💾 12. Guardar Resultados del Análisis

In [ ]:
# Crear directorio para resultados
results_dir = project_root / 'results' / 'activation_analysis'
results_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Directorio de resultados: {results_dir}")

In [ ]:
# Guardar estadísticas en CSV
stats_df.to_csv(results_dir / 'activation_statistics.csv', index=False)
print("✅ Estadísticas guardadas en activation_statistics.csv")

# Guardar información de neuronas muertas
import json

with open(results_dir / 'dead_neurons_info.json', 'w') as f:
    # Convertir a formato serializable (sin listas muy largas)
    serializable_info = {}
    for layer, info in dead_neurons_info.items():
        serializable_info[layer] = {
            'total_neurons': info['total_neurons'],
            'dead_neurons': info['dead_neurons'],
            'dead_percentage': info['dead_percentage'],
            'dead_indices_count': len(info['dead_indices'])
        }
    json.dump(serializable_info, f, indent=2)

print("✅ Información de neuronas muertas guardada en dead_neurons_info.json")

# Guardar estadísticas por clase
for layer_name, df in class_wise_stats.items():
    safe_layer_name = layer_name.replace('.', '_')
    df.to_csv(results_dir / f'class_stats_{safe_layer_name}.csv', index=False)

print(f"✅ Estadísticas por clase guardadas ({len(class_wise_stats)} archivos)")

In [ ]:
# Opcional: Guardar activaciones completas para análisis posterior
# ADVERTENCIA: Esto puede ocupar mucho espacio
SAVE_ACTIVATIONS = False

if SAVE_ACTIVATIONS:
    activations_file = results_dir / 'concatenated_activations.pth'
    torch.save({
        'activations': {k: v.cpu() for k, v in concatenated_activations.items()},
        'labels': labels_array,
        'class_names': dataset_info['classes'],
        'num_samples': len(labels_array)
    }, activations_file)
    
    file_size_mb = activations_file.stat().st_size / (1024 * 1024)
    print(f"✅ Activaciones guardadas en {activations_file}")
    print(f"📦 Tamaño del archivo: {file_size_mb:.2f} MB")
else:
    print("ℹ️  Activaciones NO guardadas (SAVE_ACTIVATIONS=False)")
    print("   Cambia a True si necesitas guardarlas para análisis futuro")

---
## 📝 13. Conclusiones y Observaciones Clave

In [ ]:
print("="*80)
print("📊 RESUMEN DEL ANÁLISIS DE ACTIVACIONES")
print("="*80)

print("\n🔍 Hallazgos Principales:\n")

# 1. Capa con mayor sparsity
max_sparsity_idx = stats_df['sparsity_%'].idxmax()
max_sparsity_layer = stats_df.loc[max_sparsity_idx, 'layer']
max_sparsity_value = stats_df.loc[max_sparsity_idx, 'sparsity_%']
print(f"1. Capa con MAYOR sparsity: {max_sparsity_layer} ({max_sparsity_value:.2f}%)")

# 2. Capa con menor sparsity
min_sparsity_idx = stats_df['sparsity_%'].idxmin()
min_sparsity_layer = stats_df.loc[min_sparsity_idx, 'layer']
min_sparsity_value = stats_df.loc[min_sparsity_idx, 'sparsity_%']
print(f"2. Capa con MENOR sparsity: {min_sparsity_layer} ({min_sparsity_value:.2f}%)")

# 3. Capa con más neuronas muertas
max_dead_layer = max(dead_neurons_info.items(), key=lambda x: x[1]['dead_percentage'])
print(f"3. Capa con MÁS neuronas muertas: {max_dead_layer[0]} ({max_dead_layer[1]['dead_percentage']:.2f}%)")

# 4. Total de neuronas muertas
total_dead = sum(info['dead_neurons'] for info in dead_neurons_info.values())
total_neurons = sum(info['total_neurons'] for info in dead_neurons_info.values())
print(f"4. Total de neuronas muertas: {total_dead}/{total_neurons} ({total_dead/total_neurons*100:.2f}%)")

# 5. Rango de medias
print(f"\n5. Rango de medias de activaciones:")
print(f"   - Mínima: {stats_df['mean'].min():.4f} ({stats_df.loc[stats_df['mean'].idxmin(), 'layer']})")
print(f"   - Máxima: {stats_df['mean'].max():.4f} ({stats_df.loc[stats_df['mean'].idxmax(), 'layer']})")

print("\n" + "="*80)
print("💡 INTERPRETACIONES:")
print("="*80)
print("""
1. SPARSITY:
   - Alta sparsity (>50%) en capas profundas es normal debido a ReLU
   - Indica selectividad: pocas neuronas se activan para cada input
   - Puede ser beneficioso para eficiencia computacional

2. NEURONAS MUERTAS:
   - Algunas neuronas pueden estar especializadas en features raros
   - Un % muy alto (>70%) puede indicar sobreajuste o dying ReLU
   - Considerar: dropout, batch normalization, learning rate

3. EVOLUCIÓN DE ACTIVACIONES:
   - Capas tempranas: features generales (bordes, texturas)
   - Capas profundas: features específicos (objetos, partes)
   - La media debería estabilizarse en capas profundas

4. DIFERENCIAS ENTRE CLASES:
   - Clases visualmente similares → activaciones similares
   - Clases muy diferentes → activaciones divergentes
   - Útil para entender confusiones del modelo
""")

print("="*80)
print("🎯 PRÓXIMOS PASOS:")
print("="*80)
print("""
✅ Completado: Análisis estadístico de activaciones

📋 Siguiente (Notebook 03): Feature Visualization
   - Generar imágenes sintéticas que maximicen activaciones
   - Visualizar qué detecta cada neurona
   - Identificar features de bajo y alto nivel

📋 Siguiente (Notebook 04): Neuron Probing
   - Entrenar clasificadores sobre activaciones
   - Entender qué información codifica cada capa
   - Análisis de emergencia de conceptos
""")

print("="*80)

---
## 🧹 14. Limpieza

In [ ]:
# Remover hooks para liberar memoria
activation_hook.remove_hooks()
print("✅ Hooks removidos")

# Limpiar cache de CUDA si es necesario
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ Cache de CUDA limpiado")

print("\n🎉 Notebook 02 completado exitosamente!")

---
## 📚 Funciones Helper (Si se necesitan agregar)

Las siguientes funciones pueden agregarse a `src/interpretability/activation_analyzer.py` si se reutilizan frecuentemente.

In [ ]:
# FUNCIONES HELPER - Copiar a src/interpretability/activation_analyzer.py si es necesario

def compute_activation_statistics(activations: Dict[str, torch.Tensor]) -> pd.DataFrame:
    """Calcula estadísticas completas de activaciones por capa."""
    # Implementación ya mostrada arriba
    pass

def find_dead_neurons_detailed(
    activations: Dict[str, torch.Tensor],
    threshold: float = 1e-6
) -> Dict[str, Dict]:
    """Identifica neuronas muertas por capa."""
    # Implementación ya mostrada arriba
    pass

def compute_class_wise_statistics(
    activations: Dict[str, torch.Tensor],
    labels: np.ndarray,
    class_names: List[str]
) -> Dict[str, pd.DataFrame]:
    """Calcula estadísticas de activaciones por clase."""
    # Implementación ya mostrada arriba
    pass

def extract_activations_from_batch(
    model: nn.Module,
    hook: ActivationHook,
    images: torch.Tensor,
    device: torch.device
) -> Dict[str, torch.Tensor]:
    """Extrae activaciones de un batch de imágenes."""
    # Implementación ya mostrada arriba
    pass

print("💡 Estas funciones pueden moverse a src/interpretability/activation_analyzer.py")
print("   para reutilización en futuros notebooks.")